In [1]:
import numpy as np
import copy

In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def derivativeSigmoid(x):
    return x*(1-x)

In [5]:
int2bin = {}
binary_dim = 8
largest_num = pow(2,binary_dim)

In [6]:
largest_num

256

In [7]:
binary = np.unpackbits(np.array([range(largest_num)],dtype=np.uint8).T,axis=1)

In [8]:
binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [9]:
binary.shape

(256, 8)

In [11]:
for i in range(largest_num):
    int2bin[i] = binary [i]

In [12]:
alpha = 0.1
input_neurons = 2
hidden_neurons = 16
output_neurons = 1

In [13]:
whidden = np.random.random((input_neurons,hidden_neurons))
wout = np.random.random((hidden_neurons,output_neurons))
wtime = np.random.random((hidden_neurons,hidden_neurons))

In [14]:
wh_update = np.zeros_like(whidden)
wo_update = np.zeros_like(wout)
wt_update = np.zeros_like(wtime)


In [ ]:
epochs=100000
for epoch in range(epochs):
    # feedforward
    a_int = np.random.randint(largest_num/2)
    a = int2bin[a_int]
    b_int = np.random.randint(largest_num/2)
    b = int2bin[b_int]
    
    c_int = a_int + b_int
    c = int2bin[c_int]
    
    d = np.zeros_like(c)
    overAllError = 0
    
    layer_1_values = list()
    layer_2_deltas  = list()
    layer_1_values.append(np.zeros(hidden_neurons))
    
    for position in range(binary_dim):
        X = np.array([[a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        Y = np.array([[c[binary_dim - position - 1]]]).T
        
        #hidden layer
        layer_1 = sigmoid(np.dot(X,whidden)+np.dot(layer_1_values[-1],wtime))
        #output layer
        layer_2 = sigmoid(np.dot(layer_1,wout))
        
        #output layer error
        layer_2_error = Y - layer_2
        slope_output = derivativeSigmoid(layer_2)
        delta_output = layer_2_error * slope_output
        layer_2_deltas.append(delta_output)
        
        overAllError += np.abs(layer_2_error[0])
        d[binary_dim - position -1 ] = np.round(layer_2_error[0][0])
        
        #storing hidden layer to use in next time step
        layer_1_values.append((copy.deepcopy(layer_1)))
        
    future_layer_1_delta =  np.zeros(hidden_neurons)
    
    for position in range(binary_dim):
        x =  np.array([[a[position-1],b[position-1]]])
        layer_1 =layer_1_values[-position-1]
        prev_layer_1 = layer_1_values[-position-2]
        
        #error at output layer
        layer_2_delta = layer_2_deltas[-position-1]
        
        #error at hidden layer
        slope_hidden = derivativeSigmoid(layer_1)
        error_hidden = (future_layer_1_delta.dot(wtime.T)+layer_2_delta.dot(wout.T))
        layer_1_delta = error_hidden*slope_hidden
        
        wo_update +=np.atleast_2d(layer_1).T.dot(layer_2_delta)
        wt_update +=np.atleast_2d(prev_layer_1).T.dot(layer_2_delta)
        wh_update +=X.T.dot(layer_1_delta)
        
    whidden +=wh_update*alpha
    wout +=wo_update*alpha
    wtime +=wt_update*alpha

    wo_update *=0
    wh_update *=0
    wt_update *=0

    if(epoch % 10000 ==0):
        print("Error",overAllError)
        print("Pred",d)
        print("True",c)
        out=0
        for index,x in enumerate(reversed(d)):
            out +=x*pow(2,index)
        print("Prediction => {} +{}={}".format(a_int,b_int,out))
        print("true => {} +{}={}".format(a_int,b_int,c_int))
        print("#"*50)

Error [3.74864723]
Pred [  0 255   0   0   0 255   0   0]
True [1 0 1 1 1 0 1 1]
Prediction => 82 +105=17340
true => 82 +105=187
##################################################
